In [2]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 10
np.set_printoptions(precision=4, suppress=True)
import json

import glob, os
import re

import subprocess

In [643]:
pathfolder = ''
# It should look something like this:
#'C:\\Users\\MSI\\Desktop\\Thesis\\NewDataSets\\pheme-rumour-scheme-dataset\\threads\\en'
# Note: This part takes the Pheme-dataset, but the path needs to end on the "en" folder in order for the program to work

annotation = pd.read_json('', dtype='int64')
#example path:
#annotation = pd.read_json('C:\\Users\\MSI\\Desktop\\Thesis\\NewDataSets\\pheme-rumour-scheme-dataset\\annotations\\en-scheme-annotations-fixed.json', dtype='int64')
#Note: In order for the annotations to work coorectly, the annotation must be "en-scheme-annotations-fixed.json"

export_destination = ''
#example path:
#'C:\\Users\\MSI\\Desktop\\Thesis\\Results\\test.tsv'
#Note: This can be anywhere, but the name must be "test.tsv"

trib_results = ''
#example path:
#'C:\\Users\\MSI\\Desktop\\Thesis\\Results\\phoeme\\Trib_results.csv'
#Note: This value takes the result of Tribrid_pos, so make sure that the name set on Tribrid_pos is the same as here

ValueError: Expected object or value

In [430]:
replyfoldername = "reactions"

sourcefoldername = "source-tweets"

structurejsonname = "structure.json"

In [432]:
#annotationkey = annotation[["tweetid","support","responsetype-vs-source","responsetype-vs-previous"]]
annotationkey = annotation.copy()
annotationkey = annotationkey.set_index('tweetid')

In [433]:
def createtwittertuplelist(structurepath):
    with open(structurejson) as f:
        #print("4")
        lines = f.readlines()
    #for i in lines:
        #print(i,"\n")

    #convert the json with regex to only contain '{' ',' numbers and '}'    
    pattern = r'|\"|[[]]|:|[[]|[]]|\\|\''
    regex = re.compile(pattern, flags=re.IGNORECASE)

    lines2 = re.sub(pattern,'',str(lines))

    #split them by numbers and non-numbers to create a tupleset of the interactions
    splitlines = re.split('(\d*.)', lines2)
    splitlines = [i for i in splitlines if i and i != splitlines[0]]

    sources =[]
    perspective = ""
    tupleset = []

    #the split string are turned into tuple-sets as follows : first number is the claim, 
    #the second is perspective the first set of tuples is always the same
    #number on both sides, to indicate that it's a new thread
#

#
    for i in range(len(splitlines)):
        #print("6")
        if not i == 0:
            #print("7")
            if i == 1:
                #print("8")
                sources.append(splitlines[i][:len(splitlines[i]) -1])
                #print("9")

            perspective = splitlines[i][:len(splitlines[i]) -1]
            tupleset.append([sources[-1], perspective])
            #print("10")

            if splitlines[i].endswith("{"):
                sources.append(splitlines[i][:len(splitlines[i]) -1])
                #print("11")
            if splitlines[i].endswith("}"):
                sources.pop()
                #print("12")
            if not perspective:
                tupleset.pop()
                #print("13")
    return(tupleset)

# Data Preparation

In [434]:
%%time
# copy of copy to put print between every line
# <<<<<<<RUN THIS

df =pd.DataFrame(
{"Label" : [], 
"x" : [], 
"y" : [],
"claim" : [], 
"perspective" : []})


label = ""
reply = ""
source = ""
unimportantcount = 0
unimportantmarks=[]

newsbookmarks=[0]
threadbookmarks = [0]
pnumber = 0
alltuples = []

#print("0:",pathfolder)
#print("0")
for trainfolders in os.listdir(pathfolder):
    trainfolder = os.path.join(pathfolder, trainfolders) #\threads\en\all the folders
    #print("1")
    #print("1:",trainfolder)
    if os.path.isdir(trainfolder):
        #print("2")
        for topicfolders in os.listdir(trainfolder): 
            allreplies = os.path.join(trainfolder, topicfolders) #\threads\en\all the folders\ threads in number-form
            #print(allreplies)
            #print(allreplies)
            repliesfolder = os.path.join(allreplies, replyfoldername) #\threads in number-form\reactions
            sourcefolder = os.path.join(allreplies, sourcefoldername)
            structurejson = os.path.join(allreplies, structurejsonname)
            #print("3")
#######     
            
            tupleset = createtwittertuplelist(structurejson)
            alltuples.append(tupleset) # save all tuplesets to access later for grading
            
            for i in range(len(tupleset)):
                #print("14")
                #############################################
                
                #if the value of the tupleset is the same, then it means it's the start of the new thread,
                #so we take the claim from the annotation (it is worded more coherently, but unnecessary)
                #and the claim from the source-tweet
                if tupleset[i][0] == tupleset[i][1]: # claim = TOPIC(charliehebdo/fergusson), perspective = THREAD(source-tweet)
                
                    #source = trainfolder
                    annotationpath = os.path.join(allreplies,"annotation.json")
                    #print(annotationpath)
                    with open(annotationpath) as f:
                        lines = f.readlines()
                    #for i in lines:
                        #print(i,"\n")
                    pattern = r'[[]]'
                    regex = re.compile(pattern, flags=re.IGNORECASE)
                    lines2 = re.sub(pattern,'[""]',str(lines))
                    
                    pattern2 = r'\\\\"|\\'
                    regex = re.compile(pattern, flags=re.IGNORECASE)
                    lines2 = re.sub(pattern2,'',str(lines))
                    
                    slines = lines2[2:-2]                     
                    midstep = json.loads(slines)
                    #print("15")
                    #print(midstep,"|",midstep['true'],"|",midstep['misinformation'])                    
                    #print(type(midstep),"|",type(str(midstep['true'])),"|",type(int(midstep['misinformation'])))
                    
                    #sometimes, both labels "true" and "misinformation" is present in the json, sometimes either or none
                    #below calculates all possibilities, and if marked as irrelevant if needed
                    if "true" in midstep and "misinformation" in midstep:
                        if str(midstep['true']) == "1" and int(midstep['misinformation']) == 0:
                            #print("16")
                            label = "1"
                        elif str(midstep['true']) == "0" and int(midstep['misinformation']) == 1:
                            #print("17")
                            label = "0"
                        else:
                            print("1 - tupleset", tupleset[1],"is irrelevant !")
                            label = "0"# this was commented out
                            unimportantmarks.append(len(df.index))# this was commented out
                            #break - this wasnt commented
               #######^^^^^^^^^######
            
                    elif "true" in midstep and "misinformation" not in midstep:
                        label = "1" if str(midstep['true']) == "1" else "0"
                        #if str(midstep['true']) == "1":
                            #label = "1"
                        #else:
                            #print("2 - tupleset", tupleset[1],"is irrelevant !")
                            #break
                    elif "misinformation" in midstep and "true" not in midstep:
                        label = "0" if int(midstep['misinformation']) == 1 else "0"
                        #if int(midstep['misinformation']) == 1:
                            #label = "0"
                        #else:
                            #print("3 - tupleset", tupleset[1],"is irrelevant !")
                            #break
                    else:
                        print("4 - tupleset", tupleset[1],"is irrelevant !")
                        label = "0"# this was commented out
                        unimportantmarks.append(len(df.index))# this was commented out
                        #break - this wasnt commented
               #######^^^^^^^^^######     
            
                    source = os.path.basename(os.path.normpath(trainfolder))
                    reply = midstep['category']
                    #print("19")
                    
                #############################################
                else: #claim = THREAD (source-tweet), perspective = TWEET(replies)   
                    #print("20")
                    for sources in os.listdir(sourcefolder):
                        sourcepath = os.path.join(sourcefolder, sources)
                        sourcejson = pd.read_json(sourcepath)
                        #print("21")
                    #check if claim in sourcefolder    
                    if tupleset[i][0] in str(sourcejson):
                        #print("[0] in sourcejson", sources)
                        #print("22")
                        source = sourcejson.loc['symbols','text'] #claim

                    #check if claim in repliesfolder
                    else:
                        #print("23")
                        magicnum = str(tupleset[i][0])
                        magicnum +=".json"
                        #print(magicnum)
                        replypath = replypath = os.path.join(repliesfolder,magicnum)                        
                        if os.path.isfile(replypath):
                            #print(replypath,"is correct!")
                            replyjson = pd.read_json(replypath)
                            source = replyjson.loc['symbols','text']
                        #print("24 but better")
                        
                        
                    #get perspective from the sourcefolder            
                    if tupleset[i][1] in str(sourcejson):
                        #print("26")
                        label = "0"
                        #print("[1] in sourcejson")
                        source = sourcejson.loc['symbols','text'] #claim
                        # get label
                        sourcenumber = tupleset[i][0]
                        if sourcesnumber in annotationkey.index:
                            #print("27")
                            stance = annotationkey.loc[sourcesnumber,"support"]                    
                        if stance == "supporting":
                            #print("28")
                            label = "1"
                        elif stance == "denying":
                            #print("29")
                            label = "0" #label 
                    #get perspective from the repliesfolder
                    else:
                       # print("30")
                        label = "0"
                        
                        magicnum = str(tupleset[i][1])
                        magicnum +=".json"
                        #print(magicnum)
                        replypath = replypath = os.path.join(repliesfolder,magicnum)                        
                        if os.path.isfile(replypath):
                            #print(replypath,"is correct!")
                            replyjson = pd.read_json(replypath)
                            reply = replyjson.loc['symbols','text']
                        #print("31 but better")
                        
                        pnumber = int(tupleset[i][1])
                        if pnumber in annotationkey.index:
                            #print("32")
                            if not pd.isnull(annotationkey.loc[pnumber, 'support']):
                                #print("33")
                                stance =  annotationkey.loc[pnumber,"support"]
                            elif not pd.isnull(annotationkey.loc[pnumber, 'responsetype-vs-previous']):
                                #print("34")
                                stance =  annotationkey.loc[pnumber,"responsetype-vs-previous"]
                            elif not pd.isnull(annotationkey.loc[pnumber, 'responsetype-vs-source']):   
                                #print("35")
                                stance =  annotationkey.loc[pnumber,"responsetype-vs-source"]
                            if stance == "supporting" or stance == "agreed":
                                #print("36")
                                label = "1"
                            elif stance == "denying" or stance == "disagreed":
                                #print("37")
                                label = "0"
                            else:
                                #print("38")
                                label = "0"
                                unimportantcount +=1
                                unimportantmarks.append(len(df.index))
                                
                #print(label, "0", "0", source, "||",reply)
                df.loc[len(df.index)] = [label, "0", "0", source, reply] ### label=0 for now, bc we don't need it
                #print("39")
                print("<<<<<<<",len(df.index), len(tupleset))
            threadbookmarks.append(len(df.index)-1)
    newsbookmarks.append(len(df.index))

<<<<<<< 1 8
<<<<<<< 2 8
<<<<<<< 3 8
<<<<<<< 4 8
<<<<<<< 5 8
<<<<<<< 6 8
<<<<<<< 7 8
<<<<<<< 8 8
<<<<<<< 9 4
<<<<<<< 10 4
<<<<<<< 11 4
<<<<<<< 12 4
<<<<<<< 13 23
<<<<<<< 14 23
<<<<<<< 15 23
<<<<<<< 16 23
<<<<<<< 17 23
<<<<<<< 18 23
<<<<<<< 19 23
<<<<<<< 20 23
<<<<<<< 21 23
<<<<<<< 22 23
<<<<<<< 23 23
<<<<<<< 24 23
<<<<<<< 25 23
<<<<<<< 26 23
<<<<<<< 27 23
<<<<<<< 28 23
<<<<<<< 29 23
<<<<<<< 30 23
<<<<<<< 31 23
<<<<<<< 32 23
<<<<<<< 33 23
<<<<<<< 34 23
<<<<<<< 35 23
<<<<<<< 36 21
<<<<<<< 37 21
<<<<<<< 38 21
<<<<<<< 39 21
<<<<<<< 40 21
<<<<<<< 41 21
<<<<<<< 42 21
<<<<<<< 43 21
<<<<<<< 44 21
<<<<<<< 45 21
<<<<<<< 46 21
<<<<<<< 47 21
<<<<<<< 48 21
<<<<<<< 49 21
<<<<<<< 50 21
<<<<<<< 51 21
<<<<<<< 52 21
<<<<<<< 53 21
<<<<<<< 54 21
<<<<<<< 55 21
<<<<<<< 56 21
<<<<<<< 57 18
<<<<<<< 58 18
<<<<<<< 59 18
<<<<<<< 60 18
<<<<<<< 61 18
<<<<<<< 62 18
<<<<<<< 63 18
<<<<<<< 64 18
<<<<<<< 65 18
<<<<<<< 66 18
<<<<<<< 67 18
<<<<<<< 68 18
<<<<<<< 69 18
<<<<<<< 70 18
<<<<<<< 71 18
<<<<<<< 72 18
<<<<<<< 73 18

<<<<<<< 531 24
<<<<<<< 532 24
<<<<<<< 533 24
<<<<<<< 534 24
<<<<<<< 535 24
<<<<<<< 536 24
<<<<<<< 537 24
<<<<<<< 538 24
<<<<<<< 539 24
<<<<<<< 540 24
<<<<<<< 541 24
<<<<<<< 542 24
<<<<<<< 543 24
1 - tupleset ['553212962044149761', '553213078889062401'] is irrelevant !
<<<<<<< 544 9
<<<<<<< 545 9
<<<<<<< 546 9
<<<<<<< 547 9
<<<<<<< 548 9
<<<<<<< 549 9
<<<<<<< 550 9
<<<<<<< 551 9
<<<<<<< 552 9
<<<<<<< 553 3
<<<<<<< 554 3
<<<<<<< 555 3
1 - tupleset ['553461741917863936', '553462142481870849'] is irrelevant !
<<<<<<< 556 20
<<<<<<< 557 20
<<<<<<< 558 20
<<<<<<< 559 20
<<<<<<< 560 20
<<<<<<< 561 20
<<<<<<< 562 20
<<<<<<< 563 20
<<<<<<< 564 20
<<<<<<< 565 20
<<<<<<< 566 20
<<<<<<< 567 20
<<<<<<< 568 20
<<<<<<< 569 20
<<<<<<< 570 20
<<<<<<< 571 20
<<<<<<< 572 20
<<<<<<< 573 20
<<<<<<< 574 20
<<<<<<< 575 20
1 - tupleset ['553467311261503488', '553468547184467969'] is irrelevant !
<<<<<<< 576 3
<<<<<<< 577 3
<<<<<<< 578 3
1 - tupleset ['553470492565602305', '553471272353402880'] is irrelevant !

<<<<<<< 1000 26
<<<<<<< 1001 26
<<<<<<< 1002 26
<<<<<<< 1003 26
<<<<<<< 1004 26
<<<<<<< 1005 26
<<<<<<< 1006 26
<<<<<<< 1007 26
<<<<<<< 1008 26
<<<<<<< 1009 26
<<<<<<< 1010 26
<<<<<<< 1011 26
<<<<<<< 1012 26
<<<<<<< 1013 26
<<<<<<< 1014 26
<<<<<<< 1015 26
<<<<<<< 1016 20
<<<<<<< 1017 20
<<<<<<< 1018 20
<<<<<<< 1019 20
<<<<<<< 1020 20
<<<<<<< 1021 20
<<<<<<< 1022 20
<<<<<<< 1023 20
<<<<<<< 1024 20
<<<<<<< 1025 20
<<<<<<< 1026 20
<<<<<<< 1027 20
<<<<<<< 1028 20
<<<<<<< 1029 20
<<<<<<< 1030 20
<<<<<<< 1031 20
<<<<<<< 1032 20
<<<<<<< 1033 20
<<<<<<< 1034 20
<<<<<<< 1035 20
<<<<<<< 1036 13
<<<<<<< 1037 13
<<<<<<< 1038 13
<<<<<<< 1039 13
<<<<<<< 1040 13
<<<<<<< 1041 13
<<<<<<< 1042 13
<<<<<<< 1043 13
<<<<<<< 1044 13
<<<<<<< 1045 13
<<<<<<< 1046 13
<<<<<<< 1047 13
<<<<<<< 1048 13
<<<<<<< 1049 35
<<<<<<< 1050 35
<<<<<<< 1051 35
<<<<<<< 1052 35
<<<<<<< 1053 35
<<<<<<< 1054 35
<<<<<<< 1055 35
<<<<<<< 1056 35
<<<<<<< 1057 35
<<<<<<< 1058 35
<<<<<<< 1059 35
<<<<<<< 1060 35
<<<<<<< 1061 35
<<<<<<< 

<<<<<<< 1469 95
<<<<<<< 1470 95
<<<<<<< 1471 95
<<<<<<< 1472 95
<<<<<<< 1473 95
<<<<<<< 1474 95
<<<<<<< 1475 95
<<<<<<< 1476 95
<<<<<<< 1477 95
<<<<<<< 1478 95
<<<<<<< 1479 95
<<<<<<< 1480 95
<<<<<<< 1481 95
<<<<<<< 1482 95
<<<<<<< 1483 95
<<<<<<< 1484 95
<<<<<<< 1485 95
<<<<<<< 1486 95
<<<<<<< 1487 95
<<<<<<< 1488 95
<<<<<<< 1489 95
<<<<<<< 1490 95
<<<<<<< 1491 95
<<<<<<< 1492 95
<<<<<<< 1493 95
<<<<<<< 1494 95
<<<<<<< 1495 95
<<<<<<< 1496 95
<<<<<<< 1497 95
<<<<<<< 1498 95
<<<<<<< 1499 95
<<<<<<< 1500 95
<<<<<<< 1501 95
<<<<<<< 1502 95
<<<<<<< 1503 95
<<<<<<< 1504 95
<<<<<<< 1505 95
<<<<<<< 1506 95
<<<<<<< 1507 95
<<<<<<< 1508 95
1 - tupleset ['500279189405433858', '500279402635067393'] is irrelevant !
<<<<<<< 1509 17
<<<<<<< 1510 17
<<<<<<< 1511 17
<<<<<<< 1512 17
<<<<<<< 1513 17
<<<<<<< 1514 17
<<<<<<< 1515 17
<<<<<<< 1516 17
<<<<<<< 1517 17
<<<<<<< 1518 17
<<<<<<< 1519 17
<<<<<<< 1520 17
<<<<<<< 1521 17
<<<<<<< 1522 17
<<<<<<< 1523 17
<<<<<<< 1524 17
<<<<<<< 1525 17
1 - tupleset [

<<<<<<< 1913 19
<<<<<<< 1914 19
<<<<<<< 1915 19
<<<<<<< 1916 19
<<<<<<< 1917 19
<<<<<<< 1918 19
<<<<<<< 1919 19
<<<<<<< 1920 19
<<<<<<< 1921 19
<<<<<<< 1922 19
<<<<<<< 1923 19
<<<<<<< 1924 19
<<<<<<< 1925 19
<<<<<<< 1926 19
<<<<<<< 1927 19
<<<<<<< 1928 19
<<<<<<< 1929 19
1 - tupleset ['500319675797209088', '500320165142495232'] is irrelevant !
<<<<<<< 1930 12
<<<<<<< 1931 12
<<<<<<< 1932 12
<<<<<<< 1933 12
<<<<<<< 1934 12
<<<<<<< 1935 12
<<<<<<< 1936 12
<<<<<<< 1937 12
<<<<<<< 1938 12
<<<<<<< 1939 12
<<<<<<< 1940 12
<<<<<<< 1941 12
1 - tupleset ['500319801344929795', '500320311997640704'] is irrelevant !
<<<<<<< 1942 17
<<<<<<< 1943 17
<<<<<<< 1944 17
<<<<<<< 1945 17
<<<<<<< 1946 17
<<<<<<< 1947 17
<<<<<<< 1948 17
<<<<<<< 1949 17
<<<<<<< 1950 17
<<<<<<< 1951 17
<<<<<<< 1952 17
<<<<<<< 1953 17
<<<<<<< 1954 17
<<<<<<< 1955 17
<<<<<<< 1956 17
<<<<<<< 1957 17
<<<<<<< 1958 17
1 - tupleset ['500327106824245249', '500327239419195392'] is irrelevant !
<<<<<<< 1959 8
<<<<<<< 1960 8
<<<<<<< 1961

...

<<<<<<< 2827 42
<<<<<<< 2828 42
<<<<<<< 2829 42
<<<<<<< 2830 42
<<<<<<< 2831 42
<<<<<<< 2832 42
<<<<<<< 2833 42
<<<<<<< 2834 42
<<<<<<< 2835 42
<<<<<<< 2836 42
<<<<<<< 2837 42
1 - tupleset ['524929497205055488', '524929713014181890'] is irrelevant !
<<<<<<< 2838 13
<<<<<<< 2839 13
<<<<<<< 2840 13
<<<<<<< 2841 13
<<<<<<< 2842 13
<<<<<<< 2843 13
<<<<<<< 2844 13
<<<<<<< 2845 13
<<<<<<< 2846 13
<<<<<<< 2847 13
<<<<<<< 2848 13
<<<<<<< 2849 13
<<<<<<< 2850 13
<<<<<<< 2851 8
<<<<<<< 2852 8
<<<<<<< 2853 8
<<<<<<< 2854 8
<<<<<<< 2855 8
<<<<<<< 2856 8
<<<<<<< 2857 8
<<<<<<< 2858 8
1 - tupleset ['524932056560963584', '524932989613256705'] is irrelevant !
<<<<<<< 2859 4
<<<<<<< 2860 4
<<<<<<< 2861 4
<<<<<<< 2862 4
<<<<<<< 2863 36
<<<<<<< 2864 36
<<<<<<< 2865 36
<<<<<<< 2866 36
<<<<<<< 2867 36
<<<<<<< 2868 36
<<<<<<< 2869 36
<<<<<<< 2870 36
<<<<<<< 2871 36
<<<<<<< 2872 36
<<<<<<< 2873 36
<<<<<<< 2874 36
<<<<<<< 2875 36
<<<<<<< 2876 36
<<<<<<< 2877 36
<<<<<<< 2878 36
<<<<<<< 2879 36
<<<<<<< 2880 36


<<<<<<< 3310 7
<<<<<<< 3311 7
<<<<<<< 3312 7
<<<<<<< 3313 7
<<<<<<< 3314 7
<<<<<<< 3315 17
<<<<<<< 3316 17
<<<<<<< 3317 17
<<<<<<< 3318 17
<<<<<<< 3319 17
<<<<<<< 3320 17
<<<<<<< 3321 17
<<<<<<< 3322 17
<<<<<<< 3323 17
<<<<<<< 3324 17
<<<<<<< 3325 17
<<<<<<< 3326 17
<<<<<<< 3327 17
<<<<<<< 3328 17
<<<<<<< 3329 17
<<<<<<< 3330 17
<<<<<<< 3331 17
<<<<<<< 3332 35
<<<<<<< 3333 35
<<<<<<< 3334 35
<<<<<<< 3335 35
<<<<<<< 3336 35
<<<<<<< 3337 35
<<<<<<< 3338 35
<<<<<<< 3339 35
<<<<<<< 3340 35
<<<<<<< 3341 35
<<<<<<< 3342 35
<<<<<<< 3343 35
<<<<<<< 3344 35
<<<<<<< 3345 35
<<<<<<< 3346 35
<<<<<<< 3347 35
<<<<<<< 3348 35
<<<<<<< 3349 35
<<<<<<< 3350 35
<<<<<<< 3351 35
<<<<<<< 3352 35
<<<<<<< 3353 35
<<<<<<< 3354 35
<<<<<<< 3355 35
<<<<<<< 3356 35
<<<<<<< 3357 35
<<<<<<< 3358 35
<<<<<<< 3359 35
<<<<<<< 3360 35
<<<<<<< 3361 35
<<<<<<< 3362 35
<<<<<<< 3363 35
<<<<<<< 3364 35
<<<<<<< 3365 35
<<<<<<< 3366 35
<<<<<<< 3367 12
<<<<<<< 3368 12
<<<<<<< 3369 12
<<<<<<< 3370 12
<<<<<<< 3371 12
<<<<<<< 3372 

<<<<<<< 3829 17
<<<<<<< 3830 17
<<<<<<< 3831 17
<<<<<<< 3832 17
<<<<<<< 3833 17
<<<<<<< 3834 17
<<<<<<< 3835 17
<<<<<<< 3836 17
<<<<<<< 3837 17
<<<<<<< 3838 17
<<<<<<< 3839 17
<<<<<<< 3840 17
<<<<<<< 3841 17
<<<<<<< 3842 17
<<<<<<< 3843 19
<<<<<<< 3844 19
<<<<<<< 3845 19
<<<<<<< 3846 19
<<<<<<< 3847 19
<<<<<<< 3848 19
<<<<<<< 3849 19
<<<<<<< 3850 19
<<<<<<< 3851 19
<<<<<<< 3852 19
<<<<<<< 3853 19
<<<<<<< 3854 19
<<<<<<< 3855 19
<<<<<<< 3856 19
<<<<<<< 3857 19
<<<<<<< 3858 19
<<<<<<< 3859 19
<<<<<<< 3860 19
<<<<<<< 3861 19
<<<<<<< 3862 15
<<<<<<< 3863 15
<<<<<<< 3864 15
<<<<<<< 3865 15
<<<<<<< 3866 15
<<<<<<< 3867 15
<<<<<<< 3868 15
<<<<<<< 3869 15
<<<<<<< 3870 15
<<<<<<< 3871 15
<<<<<<< 3872 15
<<<<<<< 3873 15
<<<<<<< 3874 15
<<<<<<< 3875 15
<<<<<<< 3876 15
<<<<<<< 3877 16
<<<<<<< 3878 16
<<<<<<< 3879 16
<<<<<<< 3880 16
<<<<<<< 3881 16
<<<<<<< 3882 16
<<<<<<< 3883 16
<<<<<<< 3884 16
<<<<<<< 3885 16
<<<<<<< 3886 16
<<<<<<< 3887 16
<<<<<<< 3888 16
<<<<<<< 3889 16
<<<<<<< 3890 16
<<<<<<< 

<<<<<<< 4329 103
<<<<<<< 4330 103
<<<<<<< 4331 103
<<<<<<< 4332 103
<<<<<<< 4333 103
<<<<<<< 4334 103
<<<<<<< 4335 103
<<<<<<< 4336 103
<<<<<<< 4337 103
<<<<<<< 4338 103
<<<<<<< 4339 103
<<<<<<< 4340 103
<<<<<<< 4341 103
<<<<<<< 4342 103
<<<<<<< 4343 103
<<<<<<< 4344 103
<<<<<<< 4345 103
<<<<<<< 4346 103
<<<<<<< 4347 103
<<<<<<< 4348 103
<<<<<<< 4349 103
<<<<<<< 4350 103
<<<<<<< 4351 103
<<<<<<< 4352 103
<<<<<<< 4353 103
<<<<<<< 4354 103
<<<<<<< 4355 103
<<<<<<< 4356 103
<<<<<<< 4357 103
<<<<<<< 4358 103
<<<<<<< 4359 103
<<<<<<< 4360 103
<<<<<<< 4361 103
<<<<<<< 4362 103
<<<<<<< 4363 103
<<<<<<< 4364 103
<<<<<<< 4365 103
<<<<<<< 4366 103
<<<<<<< 4367 103
<<<<<<< 4368 103
<<<<<<< 4369 103
<<<<<<< 4370 103
<<<<<<< 4371 103
<<<<<<< 4372 103
<<<<<<< 4373 103
<<<<<<< 4374 103
<<<<<<< 4375 103
<<<<<<< 4376 103
<<<<<<< 4377 103
<<<<<<< 4378 103
<<<<<<< 4379 103
<<<<<<< 4380 103
<<<<<<< 4381 103
<<<<<<< 4382 103
<<<<<<< 4383 103
<<<<<<< 4384 103
<<<<<<< 4385 103
<<<<<<< 4386 103
<<<<<<< 4387 1

<<<<<<< 4822 17
<<<<<<< 4823 17
<<<<<<< 4824 17
<<<<<<< 4825 17
<<<<<<< 4826 17
<<<<<<< 4827 17
<<<<<<< 4828 17
<<<<<<< 4829 17
<<<<<<< 4830 17
<<<<<<< 4831 17
<<<<<<< 4832 17
<<<<<<< 4833 17
<<<<<<< 4834 17
<<<<<<< 4835 17
<<<<<<< 4836 17
<<<<<<< 4837 17
<<<<<<< 4838 17
<<<<<<< 4839 3
<<<<<<< 4840 3
<<<<<<< 4841 3
<<<<<<< 4842 19
<<<<<<< 4843 19
<<<<<<< 4844 19
<<<<<<< 4845 19
<<<<<<< 4846 19
<<<<<<< 4847 19
<<<<<<< 4848 19
<<<<<<< 4849 19
<<<<<<< 4850 19
<<<<<<< 4851 19
<<<<<<< 4852 19
<<<<<<< 4853 19
<<<<<<< 4854 19
<<<<<<< 4855 19
<<<<<<< 4856 19
<<<<<<< 4857 19
<<<<<<< 4858 19
<<<<<<< 4859 19
<<<<<<< 4860 19
Wall time: 7min 54s


In [9]:
df

,Label,x,y,claim,perspective
0,1,0,0,charliehebdo,(At least) 10 people are dead at Charlie Hebdo...
1,0,0,0,France: 10 people dead after shooting at HQ of...,MT @euronews France: 10 dead after shooting at...
2,0,0,0,MT @euronews France: 10 dead after shooting at...,@j0nathandavis They who? Stupid and partial op...
3,1,0,0,@j0nathandavis They who? Stupid and partial op...,"@nanoSpawn Socialists, Antisemites, anti zioni..."
4,0,0,0,France: 10 people dead after shooting at HQ of...,@euronews @Massimilian2611 ci vuole una nuova ...
...,...,...,...,...,...
4855,0,0,0,Police confirm that #sydneysiege is finally ov...,@Angus_OL @horanosaur
4856,1,0,0,Police confirm that #sydneysiege is finally ov...,@Angus_OL The 6 of us are watching this unfold...
4857,1,0,0,Police confirm that #sydneysiege is finally ov...,@Angus_OL thank god they're all safe now. some...
4858,0,0,0,Police confirm that #sydneysiege is finally ov...,"@Angus_OL thank god its over, they're finally ..."


In [435]:
# regex: if '@' or 'htttp://', then delete whole string/ until theres a blank space
pattern = r'@|#|http(\S)*'
regex = re.compile(pattern, flags=re.IGNORECASE)

for i in range((len(df))):
    if (type(df['perspective'][i]) != float):
        df['perspective'][i] = regex.sub('', df['perspective'][i])
    if (type(df['claim'][i]) != float):
        df['claim'][i] = regex.sub('', df['claim'][i])

In [12]:
df

,Label,x,y,claim,perspective
0,1,0,0,charliehebdo,(At least) 10 people are dead at Charlie Hebdo...
1,0,0,0,France: 10 people dead after shooting at HQ of...,MT euronews France: 10 dead after shooting at ...
2,0,0,0,MT euronews France: 10 dead after shooting at ...,j0nathandavis They who? Stupid and partial opi...
3,1,0,0,j0nathandavis They who? Stupid and partial opi...,"nanoSpawn Socialists, Antisemites, anti zionis..."
4,0,0,0,France: 10 people dead after shooting at HQ of...,euronews Massimilian2611 ci vuole una nuova cr...
...,...,...,...,...,...
4855,0,0,0,Police confirm that sydneysiege is finally ove...,Angus_OL horanosaur
4856,1,0,0,Police confirm that sydneysiege is finally ove...,"Angus_OL The 6 of us are watching this unfold,..."
4857,1,0,0,Police confirm that sydneysiege is finally ove...,Angus_OL thank god they're all safe now. some ...
4858,0,0,0,Police confirm that sydneysiege is finally ove...,"Angus_OL thank god its over, they're finally s..."


In [13]:
df = df.reset_index(drop=True)
df

,Label,x,y,claim,perspective
0,1,0,0,charliehebdo,(At least) 10 people are dead at Charlie Hebdo...
1,0,0,0,France: 10 people dead after shooting at HQ of...,MT euronews France: 10 dead after shooting at ...
2,0,0,0,MT euronews France: 10 dead after shooting at ...,j0nathandavis They who? Stupid and partial opi...
3,1,0,0,j0nathandavis They who? Stupid and partial opi...,"nanoSpawn Socialists, Antisemites, anti zionis..."
4,0,0,0,France: 10 people dead after shooting at HQ of...,euronews Massimilian2611 ci vuole una nuova cr...
...,...,...,...,...,...
4855,0,0,0,Police confirm that sydneysiege is finally ove...,Angus_OL horanosaur
4856,1,0,0,Police confirm that sydneysiege is finally ove...,"Angus_OL The 6 of us are watching this unfold,..."
4857,1,0,0,Police confirm that sydneysiege is finally ove...,Angus_OL thank god they're all safe now. some ...
4858,0,0,0,Police confirm that sydneysiege is finally ove...,"Angus_OL thank god its over, they're finally s..."


In [436]:
#export to csv/tsv
df.to_csv(export_destination, sep = '\t', index = False)

# Tribrid_pos for stance classificaiton

In [554]:
tribrid_results = pd.read_csv(trib_results)

In [555]:
#change the 0's to -1's to better calculate
for i in range(len(tribrid_results)):
    if tribrid_results['predict_label'][i] == 0:
        tribrid_results.at[i,'predict_label'] = -1
    if tribrid_results['gold'][i] == 0:
        tribrid_results.at[i,'gold'] = -1

In [556]:
threadbookmarks2 = []
[threadbookmarks2.append(x) for x in threadbookmarks if x not in threadbookmarks2]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [557]:
flattuples = []
for i in range(len(alltuples)): #Traversing through the main list
  for j in range (len(alltuples[i])): #Traversing through each sublist
    flattuples.append(alltuples[i][j]) #Appending elements into our flat_list

newdf3 = tribrid_results.copy()
newdf3.rename(columns = {'undermine_score' : 'claim', 'support_score' : 'perspective', 'gold' : 'result'}, inplace = True)
newdf3 = newdf3.astype({"claim": str, "perspective": str})
for i in range(len(newdf3)):
    newdf3.at[i,'claim'] = flattuples[i][0]#[-5:]
    newdf3.at[i,'perspective'] = flattuples[i][1]#[-5:]
    #changing all values to thread/replynumbers

    if newdf3.at[i,'claim'] == newdf3.at[i,'perspective']:
        newdf3.at[i,'predict_label'] = newdf3.at[i,'result']
    #changing all thread "predictions" with results, as it is our guide to calculate how right/wrong people are
    
    else:
        claim_result = newdf3.index[newdf3['perspective'] == newdf3.at[i,'claim']]
        #find the claim's stance by getting the index(es)(but then getting the first instance of the index) 
        #of the claim's number in perspective column
        newdf3.at[i,'result'] = newdf3.at[claim_result[0],'result'] * newdf3.at[i,'predict_label']
    #changing the result with previous stance * this stance (written better in evernote)
    
    
#print("len of df3:", len(newdf3))    
#for i in range(len(newdf3)):
    #print(newdf3.iloc[i],"\n")
#newdf3

## Weighing

In [625]:
copy3['result'] = copy3['result'].astype(float)

In [609]:
copy3 = newdf3.copy()
for i in range((len(copy3))):
    x = int(copy3.at[i,'perspective'])
    y = int(copy3.at[i,'claim'])
    val = int(copy3.at[i,'result'])
    
    # if thread had been marked as "unimportant", label the source-tweet as 0
    if copy3.at[i,'claim'] == copy3.at[i,'perspective'] and i in unimportantmarks:
        copy3.at[i,'predict_label'] = 0
        copy3.at[i,'result'] = 0
        
        
    if x in annotationkey.index:
        #print("val:",val)
        if not x == y:
            
            #starts at no evidence w 0.5 multiplier, increases by 0.25 the more solid the evidence is
            if not pd.isnull(annotationkey.loc[x,'evidentiality']):

                if annotationkey.loc[x,'evidentiality'] == "witnessed":
                    val = val * 2

                elif annotationkey.loc[x,'evidentiality'] == "no-evidence":
                    val = val * 0.5

                elif annotationkey.loc[x,'evidentiality'] == "source-quoted":
                    val = val * 1.5

                elif annotationkey.loc[x,'evidentiality'] == "unverifiable-source-quoted":
                    val = val * 1

                elif annotationkey.loc[x,'evidentiality'] == "url-given":
                    val = val * 1.75

                elif annotationkey.loc[x,'evidentiality'] == "picture-attached":
                    val = val * 1.25

                elif annotationkey.loc[x,'evidentiality'] == "reasoning":
                    val = val * 0.75
                                            
            #print("val checkpoint 1:",annotationkey.loc[x,'evidentiality'],val)
            #check for certainty, and do a multiplier (0.5 to 2) based on it        
            if not pd.isnull(annotationkey.loc[x,'certainty']): 

                if annotationkey.loc[x,'certainty'] == "certain":
                    val = val * 2

                elif annotationkey.loc[x,'certainty'] == "somewhat-certain":
                    val = val * 1.5

                elif annotationkey.loc[x,'certainty'] == "uncertain":
                    val = val * 1

                elif annotationkey.loc[x,'certainty'] == "underspecified":
                    val = val * 0.5

            #print("val checkpoint 2:",annotationkey.loc[x,'certainty'],val)
                  
            magicnum = str(x)+".json"
            tweetpath = os.path.join(pathfolder,annotationkey.loc[x,'event'],str(annotationkey.loc[x,'threadid']),"reactions",magicnum)            
            tweetjson = pd.read_json(tweetpath)
            
            val = val + (int(copy3.at[i,'result']) * int(tweetjson.loc['friends_count','user'])*0.0001)
            #print("val checkpoint 3.1:",tweetjson.loc['friends_count','user'],val)      
            
            val = val + (int(copy3.at[i,'result']) * int(tweetjson.loc['statuses_count','user'])*0.000025)            
            #print("val checkpoint 3.2:",tweetjson.loc['statuses_count','user'],val)      
                  
            ##### need to do this #####            
            if "charliehebdo" in tweetpath and not pd.isnull(tweetjson.loc['time_zone','user']):
                if "Paris" in tweetjson.loc['time_zone','user']:
                    val = val * 1.5
                if "Europe" or "London" or "Madrid" or "Budapest" or "Amsterdam" or "Rome" in tweetjson.loc['time_zone','user']:
                    val = val * 1.25
            
            if "ebola" in tweetpath and not pd.isnull(tweetjson.loc['time_zone','user']):
                if "Doctor" or "Dr" or "dr" or "Professor" or "Prof" or "prof" in tweetjson.loc['name','user'] or tweetjson.loc['screen_name','user'] or tweetjson.loc['description','user']:
                    val = val * 1.5
            
            
            if "ferguson" in tweetpath and not pd.isnull(tweetjson.loc['time_zone','user']):
                if "Arizona" or "EST" or "America/Toronto" or "Santiago" in tweetjson.loc['time_zone','user']:
                    val = val * 1.5
            
            
            if "germanwings" in tweetpath and not pd.isnull(tweetjson.loc['time_zone','user']):
                if "Europe" or "London" or "Madrid" or "Budapest" or "Amsterdam" or "Rome" in tweetjson.loc['time_zone','user']:
                    val = val * 1.5
                if "pilot" or "Pilot" in tweetjson.loc['name','user'] or tweetjson.loc['screen_name','user'] or tweetjson.loc['description','user']:
                    val = val * 1.5
            
            if "ottawa" in tweetpath and not pd.isnull(tweetjson.loc['time_zone','user']):
                if "America/Toronto" or "Atlantic Time (Canada)" "(US & Canada)" in tweetjson.loc['time_zone','user']:
                    val = val * 1.5
                if "(US & Canada)" or "Santiago" in tweetjson.loc['time_zone','user']:
                    val = val * 1.5
            
            if "prince-toronto" in tweetpath and not pd.isnull(tweetjson.loc['time_zone','user']):
                if "America/Toronto" or "Atlantic Time (Canada)" "(US & Canada)" in tweetjson.loc['time_zone','user']:
                    val = val * 1.5
            
            
            if "putinmissing" in tweetpath and not pd.isnull(tweetjson.loc['time_zone','user']):
                if "Russia" or "Moscow" or "Yakutsk" in tweetjson.loc['time_zone','user']:
                    val = val * 1.5
            
            
            if "sydneysiege" in tweetpath and not pd.isnull(tweetjson.loc['time_zone','user']):
                if "Australia" or "Sydney" or "Melbourne" or "Brisbane" in tweetjson.loc['time_zone','user']:
                    val = val * 1.5
                    
            ###########################
            #print("val checkpoint 3.5:",tweetjson.loc['time_zone','user'],val)
            
            if tweetjson.loc['verified','user']:
                val = val*1.33
            
            #print("val checkpoint 4:",tweetjson.loc['verified','user'],val)      
            
            val = val+ (int(copy3.at[i,'result']) * (int(tweetjson.loc['favourites_count','user'])*0.005))
            #print("val checkpoint 5:", int(tweetjson.loc['favourites_count','user'])*0.005)
            
            copy3['result'] = copy3['result'].astype(float)
            copy3.at[i,'result'] = val
            #print("final val of result:", copy3.at[i,'result'])
            #print(tweetjson)    

KeyboardInterrupt: 

In [626]:
### WITHOUT EVIDENTIALITY OR CREDABILITY
copy3 = newdf3.copy()
for i in range((len(copy3))):
    x = int(copy3.at[i,'perspective'])
    y = int(copy3.at[i,'claim'])
    val = int(copy3.at[i,'result'])
    
    # if thread had been marked as "unimportant", label the source-tweet as 0
    if copy3.at[i,'claim'] == copy3.at[i,'perspective'] and i in unimportantmarks:
        copy3.at[i,'predict_label'] = 0
        copy3.at[i,'result'] = 0
        
        
    if x in annotationkey.index:
        print("val:",val)
        if not x == y:
                  
            ##### need to do this #####            
            if "charliehebdo" in tweetpath and not pd.isnull(tweetjson.loc['time_zone','user']):
                if "Paris" in tweetjson.loc['time_zone','user']:
                    val = val * 1.5
                if "Europe" or "London" or "Madrid" or "Budapest" or "Amsterdam" or "Rome" in tweetjson.loc['time_zone','user']:
                    val = val * 1.25
            
            if "ebola" in tweetpath and not pd.isnull(tweetjson.loc['time_zone','user']):
                if "Doctor" or "Dr" or "dr" or "Professor" or "Prof" or "prof" in tweetjson.loc['name','user'] or tweetjson.loc['screen_name','user'] or tweetjson.loc['description','user']:
                    val = val * 1.5
            
            
            if "ferguson" in tweetpath and not pd.isnull(tweetjson.loc['time_zone','user']):
                if "Arizona" or "EST" or "America/Toronto" or "Santiago" in tweetjson.loc['time_zone','user']:
                    val = val * 1.5
            
            
            if "germanwings" in tweetpath and not pd.isnull(tweetjson.loc['time_zone','user']):
                if "Europe" or "London" or "Madrid" or "Budapest" or "Amsterdam" or "Rome" in tweetjson.loc['time_zone','user']:
                    val = val * 1.5
                if "pilot" or "Pilot" in tweetjson.loc['name','user'] or tweetjson.loc['screen_name','user'] or tweetjson.loc['description','user']:
                    val = val * 1.5
            
            if "ottawa" in tweetpath and not pd.isnull(tweetjson.loc['time_zone','user']):
                if "America/Toronto" or "Atlantic Time (Canada)" "(US & Canada)" in tweetjson.loc['time_zone','user']:
                    val = val * 1.5
                if "(US & Canada)" or "Santiago" in tweetjson.loc['time_zone','user']:
                    val = val * 1.5
            
            if "prince-toronto" in tweetpath and not pd.isnull(tweetjson.loc['time_zone','user']):
                if "America/Toronto" or "Atlantic Time (Canada)" "(US & Canada)" in tweetjson.loc['time_zone','user']:
                    val = val * 1.5
            
            
            if "putinmissing" in tweetpath and not pd.isnull(tweetjson.loc['time_zone','user']):
                if "Russia" or "Moscow" or "Yakutsk" in tweetjson.loc['time_zone','user']:
                    val = val * 1.5
            
            
            if "sydneysiege" in tweetpath and not pd.isnull(tweetjson.loc['time_zone','user']):
                if "Australia" or "Sydney" or "Melbourne" or "Brisbane" in tweetjson.loc['time_zone','user']:
                    val = val * 1.5
                    
            ###########################
            print("val checkpoint 3.5:",tweetjson.loc['time_zone','user'],val)
            
            if tweetjson.loc['verified','user']:
                val = val*1.33
            
            print("val checkpoint 4:",tweetjson.loc['verified','user'],val)      
            
            val = val+ (int(copy3.at[i,'result']) * (int(tweetjson.loc['favourites_count','user'])*0.005))
            print("val checkpoint 5:", int(tweetjson.loc['favourites_count','user'])*0.005)
            
            copy3['result'] = copy3['result'].astype(float)
            copy3.at[i,'result'] = val
            print("final val of result:", copy3.at[i,'result'])
            #print(tweetjson)    

val: 1
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1

final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: Fals

final val of result: -46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: -1
val checkpoint 3.5: None -1
val checkpo

val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val

val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of res

...

val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5

final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
v

final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
v

val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 0
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkp

val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: -1
val checkpoint 3.5: None -1
val checkpoint 4: False -1
val checkpoint 5: 45.415
final val of result: -46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415
final val of result: 46.415
val: 1
val checkpoint 3.5: None 1
val checkpoint 4: False 1
val checkpoint 5: 45.415

In [611]:
### WITHOUT TWITTER API
copy3 = newdf3.copy()
for i in range((len(copy3))):
    x = int(copy3.at[i,'perspective'])
    y = int(copy3.at[i,'claim'])
    val = int(copy3.at[i,'result'])
    
    # if thread had been marked as "unimportant", label the source-tweet as 0
    if copy3.at[i,'claim'] == copy3.at[i,'perspective'] and i in unimportantmarks:
        copy3.at[i,'predict_label'] = 0
        copy3.at[i,'result'] = 0
        
        
    if x in annotationkey.index:
        print("val:",val)
        if not x == y:
            
            #starts at no evidence w 0.5 multiplier, increases by 0.25 the more solid the evidence is
            if not pd.isnull(annotationkey.loc[x,'evidentiality']):

                if annotationkey.loc[x,'evidentiality'] == "witnessed":
                    val = val * 2

                elif annotationkey.loc[x,'evidentiality'] == "no-evidence":
                    val = val * 0.5

                elif annotationkey.loc[x,'evidentiality'] == "source-quoted":
                    val = val * 1.5

                elif annotationkey.loc[x,'evidentiality'] == "unverifiable-source-quoted":
                    val = val * 1

                elif annotationkey.loc[x,'evidentiality'] == "url-given":
                    val = val * 1.75

                elif annotationkey.loc[x,'evidentiality'] == "picture-attached":
                    val = val * 1.25

                elif annotationkey.loc[x,'evidentiality'] == "reasoning":
                    val = val * 0.75
                                            
            print("val checkpoint 1:",annotationkey.loc[x,'evidentiality'],val)
            #check for certainty, and do a multiplier (0.5 to 2) based on it        
            if not pd.isnull(annotationkey.loc[x,'certainty']): 

                if annotationkey.loc[x,'certainty'] == "certain":
                    val = val * 2

                elif annotationkey.loc[x,'certainty'] == "somewhat-certain":
                    val = val * 1.5

                elif annotationkey.loc[x,'certainty'] == "uncertain":
                    val = val * 1

                elif annotationkey.loc[x,'certainty'] == "underspecified":
                    val = val * 0.5

                else:
                    print(x,i,annotationkey.loc[x,'certainty'])
            
            copy3['result'] = copy3['result'].astype(float)
            copy3.at[i,'result'] = val
            print("final val of result:", copy3.at[i,'result'])
            #print(tweetjson)    

val: 1
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: -1
val checkpoint 1: witnessed -2
final val of result: -4.0
val: -1
val checkpoint 1: no-evidence -0.5
final val of result: -1.0
val: 1
val checkpoint 1: source-quoted 1.5
final val of result: 2.25
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val: 1
val checkpoint 1: no-evidence 0.5
final val of result: 1.0
val: -1
val checkpoint 1: unverifiable-source-quoted -1
final val of result: -1.5
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: picture-attached 1.25
final val of result: 1.875
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: -1
val checkpoint 1: unverifiable-source-quoted -1
final val of result: -1.5
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: -1
val checkpoint 1: source-quoted -1.5
final val of result: -3.0


final val of result: -3.0
val: -1
val checkpoint 1: no-evidence -0.5
final val of result: -1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: witnessed 2
final val of result: 4.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: witnessed 2
final val of result: 4.0
val: -1
val checkpoint 1: witnessed -2
final val of result: -4.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: -1
val checkpoint 1: no-evidence -0.5
final val of result: -1.0
val: 1
val checkpoint 1: witnessed 2
final val of result: 4.0
val: -1
val checkpoint 1: witnessed -2
final val of result: -4.0
val: -1
val checkpoint 1: witnessed -2
final val of result: -4.0
val: 1
val checkpoint 1: no-evidence 0.5
final val of result: 1.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: 1
val checkpoint 1: no-evidence 0.5


val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkp

val checkpoint 1: witnessed -2
final val of result: -4.0
val: -1
val checkpoint 1: witnessed -2
final val of result: -4.0
val: -1
val checkpoint 1: witnessed -2
final val of result: -4.0
val: -1
val checkpoint 1: witnessed -2
final val of result: -4.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: -1
val checkpoint 1: url-given -1.75
final val of result: -3.5
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: -1
val checkpoint 1: witnessed -2
final val of result: -4.0
val: -1
val checkpoint 1: witnessed -2
final val of result: -4.0
val: 1
val checkpoint 1: url-given 1.75
final val of result: 3.5
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: -1
val chec

val checkpoint 1: no-evidence -0.5
final val of result: -0.5
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: 1
val: 1
val checkpoint 1: witnessed 2
final val of result: 4.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: -1
val checkpoint 1: no-evidence -0.5
final val of result: -0.75
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: -1
val checkpoint 1: witnessed -2
final val of result: -4.0
val: 1
val checkpoint 1: witnessed 2
final val of result: 2.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: -1
val checkpoint 1: nan -1
final val of

...

val checkpoint 1: unverifiable-source-quoted 1
final val of result: 2.0
val: -1
val checkpoint 1: picture-attached -1.25
final val of result: -1.875
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: 1
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: no-evidence 0.5
final val of result: 0.75
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: nan 1
final 

val checkpoint 1: nan -1
final val of result: -1.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: -1
val checkpoint 1: no-evidence -0.5
final val of result: -1.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: -1
val checkpoint 1: unverifiable-source-quoted -1
final val of result: -2.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: -1
val checkpoint 1: no-evidence -0.5
final val of result: -1.0
val: -1
val checkpoint 1: url-given -1.75
final val of result: -3.5
val: 1
val checkpoint 1: url-given 1.75
final val of result: 3.5
val: 1
val checkpoint 1: no-evidence 0.5
final val of result: 0.5
val: -1
val checkpoint 1: reasoning -0.75
final val of result: -1.5
val: 1
val checkpoint 1: witnessed 2
final val of resul

val: 1
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: source-quoted 1.5
final val of result: 1.5
val: 1
val: -1
val checkpoint 1: witnessed -2
final val of result: -4.0
val: -1
val checkpoint 1: no-evidence -0.5
final val of result: -1.0
val: 1
val checkpoint 1: url-given 1.75
final val of result: 3.5
val: 1
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: source-quoted 1.5
final val of result: 2.25
val: 1
val checkpoint 1: url-given 1.75
final val of result: 2.625
val: 1
val checkpoint 1: source-quoted 1.5
final val of result: 3.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: url-given 1.75
final val of result: 0.875
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: unverifiable-source-quoted 1
final val of result: 1.0
val: 1
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoi

final val of result: -1.0
val: -1
val checkpoint 1: no-evidence -0.5
final val of result: -1.0
val: -1
val checkpoint 1: no-evidence -0.5
final val of result: -1.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: 1
val checkpoint 1: no-evidence 0.5
final val of result: 1.0
val: 1
val checkpoint 1: no-evidence 0.5
final val of result: 1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: no-evidence 0.5
final val of result: 0.75
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: -1
val checkpoint 1: url-given -1.75
final val of result: -2.625
val: 1
val: -1
val checkpoint 1: source-quoted -1.5
final val of result: -1.5
val: -1
val checkpoint 1: 

final val of result: 1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val: 1
val checkpoint 1: witnessed 2
final val of result: 4.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: witnessed 2
final val of result: 4.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: witnessed 2
final val of result: 3.0
val: -1
val checkpoint 1: nan -1
final val of result: -1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: -1
val checkpoint 1: witnessed -2
final val of result: -4.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: 1
val: 1
val checkpoint 1: nan 1
final val of result: 1.0
val: -1
val checkpoint 1:

In [627]:
for i in range((len(newdf3))):
    # if thread had been marked as "unimportant", label the source-tweet as 0
    if newdf3.at[i,'claim'] == newdf3.at[i,'perspective'] and i in unimportantmarks:
        newdf3.at[i,'predict_label'] = 0
        newdf3.at[i,'result'] = 0   

In [628]:
soft3 = copy3.copy()
softness = 0.5
for i in range(len(soft3)):
    if not soft3.at[i,'claim'] == soft3.at[i,'perspective']:
        newdf3['result'] = newdf3['result'].astype(float)
        soft3.at[i,'result'] = newdf3.at[i,'result'] + (copy3.at[i,'result']/softness)
        print(soft3.at[i,'result'])

-93.83
-93.83
-93.83
3.0
3.0
93.83
93.83
93.83
-93.83
93.83
93.83
93.83
-93.83
93.83
-93.83
-93.83
-93.83
93.83
-93.83
93.83
93.83
-93.83
93.83
93.83
-93.83
93.83
93.83
-93.83
93.83
-93.83
93.83
93.83
93.83
-93.83
93.83
93.83
93.83
-93.83
93.83
93.83
93.83
93.83
93.83
93.83
-93.83
93.83
93.83
93.83
93.83
93.83
93.83
93.83
-93.83
93.83
93.83
93.83
-93.83
93.83
93.83
-93.83
-93.83
-93.83
-93.83
-93.83
93.83
93.83
-93.83
93.83
-93.83
93.83
93.83
93.83
-93.83
93.83
3.0
93.83
93.83
93.83
-93.83
93.83
93.83
93.83
93.83
93.83
3.0
-3.0
-93.83
93.83
-3.0
93.83
3.0
93.83
-93.83
93.83
93.83
93.83
93.83
93.83
-93.83
93.83
93.83
93.83
-93.83
93.83
93.83
93.83
-93.83
93.83
-93.83
93.83
93.83
93.83
-93.83
93.83
93.83
93.83
93.83
93.83
-93.83
93.83
-93.83
-93.83
-93.83
-93.83
93.83
93.83
93.83
93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
93.83
-93.83
93.83
93.83
93.83
93.83
93.83
93.83
-93.83
-93.83
-93.83
93.83
-93.83
-93.83
93.83
93.83
93.83
93.83
93.83
-93.83
93.83
93.83
93.83
-93.83
-93.83
-93.

-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
93.83
93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
93.83
-93.83
93.83
93.83
93.83
-93.83
-93.83
93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
93.83
-93.83
-93.83
-93.83
-93.83
93.83
93.83
-93.83
-93.83
93.83
-93.83
93.83
-93.83
-93.83
-93.83
-93.83
93.83
-93.83
93.83
-93.83
-93.83
-93.83
-93.83
-93.83
93.83
93.83
-93.83
-93.83
-93.83
93.83
93.83
-93.83
93.83
93.83
93.83
93.83
93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
93.83
93.83
93.83
93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
93.83
93.83
-93.83
-93.83
-93.83
-93.83
-93.83
93.83
-93.83
-93.83
93.83
-93.83
-93.83
-93.83
-93.83
-93.83
93.83
93.83
-93.83
-93.83
-93.83
93.83
93.83
93.83
93.83
93.83
93.83
93.83
93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-3.0
-3.0
93.83
93.83
-93.83
-93.83
-93.83
93.83
93.83
93.83
93.83
93.83
93.83
93.83
93.83
-93.83
93.83
93.83
93.83
93.83
93.83
93.83
-93.83
-93.

-93.83
-93.83
93.83
93.83
93.83
93.83
93.83
93.83
3.0
93.83
-93.83
93.83
-93.83
93.83
93.83
93.83
93.83
-93.83
93.83
93.83
-93.83
93.83
-93.83
-93.83
-93.83
93.83
93.83
93.83
93.83
93.83
93.83
-93.83
-93.83
-93.83
93.83
-93.83
93.83
-93.83
-93.83
-93.83
93.83
-93.83
-93.83
93.83
93.83
-93.83
93.83
-93.83
-93.83
93.83
93.83
3.0
93.83
93.83
-93.83
93.83
93.83
-93.83
93.83
-93.83
93.83
-93.83
93.83
-93.83
93.83
93.83
-93.83
93.83
93.83
3.0
93.83
-93.83
3.0
93.83
93.83
93.83
-93.83
-93.83
93.83
-93.83
3.0
3.0
3.0
3.0
3.0
3.0
93.83
93.83
-93.83
93.83
-93.83
93.83
-93.83
-93.83
93.83
-93.83
93.83
-93.83
-93.83
-93.83
-93.83
93.83
93.83
93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
93.83
-93.83
-93.83
93.83
-93.83
-93.83
-93.83
-93.83
-93.83
-93.83
93.83
-93.83
-93.83
-93.83
-93.83
-93.83
93.83
-93.83
93.83
-93.83
-93.83
93.83
93.83
93.83
-93.83
-93.83
-93.83
93.83
-93.83
-93.83
-93.83
-93.83
-93.83
93.83
-9

In [614]:
finaldf = soft3.copy()

In [615]:
#separate reply point system by thread 
elist = []
telist = []
copyforllist = []
llist = []
for i in range(len(finaldf)):
    if finaldf.at[i,'claim'] == finaldf.at[i,'perspective']: #if these numbers are the same, then we have a new thread        
        if i == 0:
            a =(finaldf.at[i,'claim']) #add threadnumber
            telist.append(finaldf.at[i,'result']) #append value to a list
            
        elif i in newsbookmarks:
            aa = tuple((a,telist[:])) #add threadnumber and list with its values to final list
            elist.append(aa)          #add threadnumber and list with its values to final list
            copyforllist.append(aa)
            llist.append(copyforllist[:])
            telist.clear()            #clear temporary list
            copyforllist.clear()
            a =(finaldf.at[i,'claim']) #add threadnumber
            telist.append(finaldf.at[i,'result']) #append value to a list 
            
        else:
            aa = tuple((a,telist[:])) #add threadnumber and list with its values to final list
            elist.append(aa)          #add threadnumber and list with its values to final list
            copyforllist.append(aa)
            telist.clear()            #clear temporary list
            a =(finaldf.at[i,'claim']) #add threadnumber
            telist.append(finaldf.at[i,'result']) #append value to a list
            
    else:
        telist.append(finaldf.at[i,'result']) #append value to a list
aa = tuple((a,telist[:]))
elist.append(aa)
copyforllist.append(aa)
llist.append(copyforllist)
#del elist[0]
#print(len(elist))
#print(elist)

In [616]:
tuplelist = []
for i in range(len(elist)): # [1,-1,-1,-1,1,1,1
    tuple1=tuple((elist[i][0],elist[i][1][0],elist[i][1][1:len(elist)]))
    tuplelist.append(tuple1)

    #divide the tuple into the following:
    #threadnumber, result(if news is true/false), reply-values(1,-1,1.. etc)
    
#for i in range(len(tuplelist)):
#    print(i,tuplelist[i])

## Showcasing Results

In [617]:
counttuple = []
forc = 0
against = 0

df5 =pd.DataFrame(
{"threadnumber" : [],
"news" : [], 
"for" : [], 
"against" : [],
"for%" : [], 
"against%" : [], 
"verdict" : []})

for i in tuplelist:
    #print(i[2])
    for j in range(len(i[2])):
        if (i[2][j] > 0):
            forc += 1
        else:
            against += 1
        #print(i[1][j])
        
    #print(forc,against)
    
    forp = forc/(forc+against)
    againstp = against/(forc+against)
    summed = sum(i[2])
    
    if i[1] > 0 and summed > 0:
        verd = "TP"   #pregnant woman is pregnant
    elif i[1] > 0 and summed < 0:
        verd = "FP"   #pregnant woman is told shes not preg
    elif i[1] < 0 and summed > 0:
        verd = "FN"   #dentist says man is pregnant
    elif i[1] < 0 and summed < 0:
        verd = "TN"   #man is told he is NOT pregnant
    elif i[1] > 0 and summed == 0:
        verd = "UP"   #pregnant woman is by equal amount that she is and is not pregnant
    elif i[1] < 0 and summed == 0:
        verd = "UN"   #man is by equal amount that she is and is not pregnant
    elif i[1] == 0 and summed < 0:
        verd = "TQ"   #true query/comment
    elif i[1] == 0 and summed > 1:
        verd = "FQ"   #false query/comment
    elif i[1] == 0 and summed == 0:
        verd = "UQ"
        
    df5.loc[len(df5.index)] = [i[0], i[1], forc, against, forp, againstp, verd]   
    #print(len(df5))
    forc = 0
    against = 0

In [618]:
reslist = []
df6 = df5.copy()
print(df6)
for i in range(len(llist)):
    print(i, len(llist[i]))
    reslist.append(df6.iloc[0:len(llist[i])])
    df6 = df6.iloc[len(llist[i]):,:]
    print("ln",len(df6))
display(reslist)

           threadnumber  news   for  against      for%  against% verdict
0    552783667052167168   1.0   4.0      3.0  0.571429  0.428571      FP
1    552785375161499649   1.0   2.0      1.0  0.666667  0.333333      TP
2    552791196247269378   1.0  13.0      9.0  0.590909  0.409091      TP
3    552791578893619200   1.0  17.0      3.0  0.850000  0.150000      TP
4    552792544132997121   1.0   8.0      9.0  0.470588  0.529412      FP
5    552792802309181440   1.0   7.0      1.0  0.875000  0.125000      TP
6    552792913910833152   1.0   4.0      1.0  0.800000  0.200000      TP
7    552793679082311680   1.0  11.0      4.0  0.733333  0.266667      TP
8    552802654641225728   1.0   5.0      2.0  0.714286  0.285714      TP
9    552805488631758849   1.0  46.0     46.0  0.500000  0.500000      FP
10   552806309540528128  -1.0  10.0     29.0  0.256410  0.743590      TN
11   552806757672964097   1.0   8.0      4.0  0.666667  0.333333      TP
12   552810448324943872   0.0   1.0     17.0  0.055

[          threadnumber  news   for  against      for%  against% verdict
 0   552783667052167168   1.0   4.0      3.0  0.571429  0.428571      FP
 1   552785375161499649   1.0   2.0      1.0  0.666667  0.333333      TP
 2   552791196247269378   1.0  13.0      9.0  0.590909  0.409091      TP
 3   552791578893619200   1.0  17.0      3.0  0.850000  0.150000      TP
 4   552792544132997121   1.0   8.0      9.0  0.470588  0.529412      FP
 5   552792802309181440   1.0   7.0      1.0  0.875000  0.125000      TP
 6   552792913910833152   1.0   4.0      1.0  0.800000  0.200000      TP
 7   552793679082311680   1.0  11.0      4.0  0.733333  0.266667      TP
 8   552802654641225728   1.0   5.0      2.0  0.714286  0.285714      TP
 9   552805488631758849   1.0  46.0     46.0  0.500000  0.500000      FP
 10  552806309540528128  -1.0  10.0     29.0  0.256410  0.743590      TN
 11  552806757672964097   1.0   8.0      4.0  0.666667  0.333333      TP
 12  552810448324943872   0.0   1.0     17.0  0.055

In [619]:
index = 0

allresulttables = []

for i in reslist:
    resulttable = pd.DataFrame(
    {"T" : [0,0,0],
     "F" : [0,0,0],
     "Q" : [0,0,0]}, 
    index = ["P", "N", "U"])
    
    TP = (i.verdict == 'TP').sum()
    FP = (i.verdict == 'FP').sum()
    TN = (i.verdict == 'TN').sum()
    FN = (i.verdict == 'FN').sum()
    UP = (i.verdict == 'UP').sum()
    UN = (i.verdict == 'UN').sum()
    TQ = (i.verdict == 'TQ').sum()
    FQ = (i.verdict == 'FQ').sum()
    UQ = (i.verdict == 'UQ').sum()
    
    resulttable.iloc[0][0] = TP
    resulttable.iloc[1][0] = FP
    resulttable.iloc[2][0] = UP
    resulttable.iloc[0][1] = TN
    resulttable.iloc[1][1] = FN
    resulttable.iloc[2][1] = UN
    resulttable.iloc[0][2] = TQ
    resulttable.iloc[1][2] = FQ
    resulttable.iloc[2][2] = UQ    
        
    allresulttables.append(resulttable)
#allresulttables   

In [620]:
for i in allresulttables:
    display(i)

,T,F,Q
P,33,10,18
N,7,2,2
U,0,0,2


,T,F,Q
P,0,2,0
N,0,0,0
U,0,0,0


,T,F,Q
P,2,0,38
N,0,0,6
U,0,0,0


,T,F,Q
P,8,9,3
N,2,3,0
U,0,0,0


,T,F,Q
P,32,6,11
N,3,4,2
U,0,0,0


,T,F,Q
P,0,11,0
N,0,1,0
U,0,0,0


,T,F,Q
P,0,7,0
N,0,1,0
U,0,1,0


,T,F,Q
P,37,10,5
N,11,3,2
U,3,0,0


In [621]:
index = 0

allpercentresulttables = []

for i in reslist:
    resulttable = pd.DataFrame(
    {"T" : [0,0,0],
     "F" : [0,0,0],
     "Q" : [0,0,0]}, 
    index = ["P", "N", "U"])
    
    TP = ((i.verdict == 'TP').sum() * 100.)/len(i)
    FP = ((i.verdict == 'FP').sum() * 100.)/len(i)
    TN = ((i.verdict == 'TN').sum() * 100.)/len(i)
    FN = ((i.verdict == 'FN').sum() * 100.)/len(i)
    UP = ((i.verdict == 'UP').sum() * 100.)/len(i)
    UN = ((i.verdict == 'UN').sum() * 100.)/len(i)
    TQ = ((i.verdict == 'TQ').sum() * 100.)/len(i)
    FQ = ((i.verdict == 'FQ').sum() * 100.)/len(i)
    UQ = ((i.verdict == 'UQ').sum() * 100.)/len(i)
    
    resulttable.iloc[0][0] = TP
    resulttable.iloc[1][0] = FP
    resulttable.iloc[2][0] = UP
    resulttable.iloc[0][1] = TN
    resulttable.iloc[1][1] = FN
    resulttable.iloc[2][1] = UN
    resulttable.iloc[0][2] = TQ
    resulttable.iloc[1][2] = FQ
    resulttable.iloc[2][2] = UQ    
        
    allpercentresulttables.append(resulttable)
#allresulttables   

In [622]:
for i in allpercentresulttables:
    display(i)

,T,F,Q
P,44,13,24
N,9,2,2
U,0,0,2


,T,F,Q
P,0,100,0
N,0,0,0
U,0,0,0


,T,F,Q
P,4,0,82
N,0,0,13
U,0,0,0


,T,F,Q
P,32,36,12
N,8,12,0
U,0,0,0


,T,F,Q
P,55,10,18
N,5,6,3
U,0,0,0


,T,F,Q
P,0,91,0
N,0,8,0
U,0,0,0


,T,F,Q
P,0,77,0
N,0,11,0
U,0,11,0


,T,F,Q
P,52,14,7
N,15,4,2
U,4,0,0


In [623]:
index = 0

alltogetherresulttables = []

resulttable = pd.DataFrame(
    {"T" : [0,0,0],
     "F" : [0,0,0],
     "Q" : [0,0,0]}, 
    index = ["P", "N", "U"])

TP = 0
FP = 0
TN = 0
FN = 0
UP = 0
UN = 0
TQ = 0
FQ = 0
UQ = 0
    
for i in reslist:    
    TP = TP+(i.verdict == 'TP').sum()
    FP = FP+(i.verdict == 'FP').sum()
    TN = TN+(i.verdict == 'TN').sum()
    FN = FN+(i.verdict == 'FN').sum()
    UP = UP+(i.verdict == 'UP').sum()
    UN = UN+(i.verdict == 'UN').sum()
    TQ = TQ+(i.verdict == 'TQ').sum()
    FQ = FQ+(i.verdict == 'FQ').sum()
    UQ = UQ+(i.verdict == 'UQ').sum()
    
    resulttable.iloc[0][0] = TP
    resulttable.iloc[1][0] = FP
    resulttable.iloc[2][0] = UP
    resulttable.iloc[0][1] = TN
    resulttable.iloc[1][1] = FN
    resulttable.iloc[2][1] = UN
    resulttable.iloc[0][2] = TQ
    resulttable.iloc[1][2] = FQ
    resulttable.iloc[2][2] = UQ    
        
alltogetherresulttables.append(resulttable)
display(alltogetherresulttables[0])

,T,F,Q
P,112,55,75
N,23,14,12
U,3,1,2


In [624]:
index = 0

alltogetherpercentresulttables = []

resulttable = pd.DataFrame(
    {"T" : [0,0,0],
     "F" : [0,0,0],
     "Q" : [0,0,0]}, 
    index = ["P", "N", "U"])

TP = 0
FP = 0
TN = 0
FN = 0
UP = 0
UN = 0
TQ = 0
FQ = 0
UQ = 0
    
for i in reslist:    
    TP = ((i.verdict == 'TP').sum() * 100.)/len(i)
    FP = ((i.verdict == 'FP').sum() * 100.)/len(i)
    TN = ((i.verdict == 'TN').sum() * 100.)/len(i)
    FN = ((i.verdict == 'FN').sum() * 100.)/len(i)
    UP = ((i.verdict == 'UP').sum() * 100.)/len(i)
    UN = ((i.verdict == 'UN').sum() * 100.)/len(i)
    TQ = ((i.verdict == 'TQ').sum() * 100.)/len(i)
    FQ = ((i.verdict == 'FQ').sum() * 100.)/len(i)
    UQ = ((i.verdict == 'UQ').sum() * 100.)/len(i)
    
    resulttable.iloc[0][0] = TP
    resulttable.iloc[1][0] = FP
    resulttable.iloc[2][0] = UP
    resulttable.iloc[0][1] = TN
    resulttable.iloc[1][1] = FN
    resulttable.iloc[2][1] = UN
    resulttable.iloc[0][2] = TQ
    resulttable.iloc[1][2] = FQ
    resulttable.iloc[2][2] = UQ    
        
alltogetherpercentresulttables.append(resulttable)
display(alltogetherpercentresulttables[0])

,T,F,Q
P,52,14,7
N,15,4,2
U,4,0,0


In [641]:
yf =pd.DataFrame(
{"Label" : [], 
"claim" : [], 
"perspective" : []})
for i in range(len(alltuples[4])):
    print(alltuples[4][i][0],alltuples[4][i][1])
    if i == 0: 
        yf.loc[len(yf.index)] = ["1", alltuples[4][i][0], alltuples[4][i][1]]
    else:
        yf.loc[len(yf.index)] = ["0", alltuples[4][i][0], alltuples[4][i][1]]

552792544132997121 552792544132997121
552792544132997121 552792742842363905
552792544132997121 552794125167493120
552792544132997121 552795993553764354
552795993553764354 552798917541511169
552798917541511169 552805237510385664
552805237510385664 552805439474515968
552805237510385664 552805557896482816
552805557896482816 552805909425307648
552805557896482816 552806009157464064
552805557896482816 552806259767123968
552806259767123968 552806489363337217
552806259767123968 552808594228670465
552805557896482816 552806438629048320
552806438629048320 552806661149425664
552805237510385664 552816964436230144
552795993553764354 552798994456674304
552792544132997121 552800910670569472


In [642]:
display(yf)

,Label,claim,perspective
0,1,552792544132997121,552792544132997121
1,0,552792544132997121,552792742842363905
2,0,552792544132997121,552794125167493120
3,0,552792544132997121,552795993553764354
4,0,552795993553764354,552798917541511169
5,0,552798917541511169,552805237510385664
6,0,552805237510385664,552805439474515968
7,0,552805237510385664,552805557896482816
8,0,552805557896482816,552805909425307648
9,0,552805557896482816,552806009157464064
